In [3]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import re
from scipy import stats
from matplotlib import style
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.dates as mdates 

# from sklearn.impute import KNNImputer
import statsmodels.formula.api as smf
import statsmodels.api as sm
from linearmodels.iv import IV2SLS
from linearmodels.iv.absorbing import AbsorbingLS

pd.set_option('display.max_columns', 100)
style.use("fivethirtyeight")

import csv
import os

######################################


## region

In [56]:
files = [] 
for dirname, _, filenames in os.walk('./archive/'):
    for filename in filenames:
        # print(os.path.join(dirname, filename))
        files.append(os.path.join(dirname, filename))

In [57]:
import random

print(len(files))
# files = random.shuffle(files)
files[:10]

278


['./archive/0819_UkraineCombinedTweetsDeduped.csv.gzip',
 './archive/0820_UkraineCombinedTweetsDeduped.csv.gzip',
 './archive/0821_UkraineCombinedTweetsDeduped.csv.gzip',
 './archive/0822_UkraineCombinedTweetsDeduped.csv.gzip',
 './archive/0823_UkraineCombinedTweetsDeduped.csv.gzip',
 './archive/0824_UkraineCombinedTweetsDeduped.csv.gzip',
 './archive/0825_UkraineCombinedTweetsDeduped.csv.gzip',
 './archive/0826_UkraineCombinedTweetsDeduped.csv.gzip',
 './archive/0827_UkraineCombinedTweetsDeduped.csv.gzip',
 './archive/0828_UkraineCombinedTweetsDeduped.csv.gzip']

In [60]:
fff = [files[0],files[10],files[30],files[70],files[90],files[100],files[110],files[120],files[130],files[150],\
       files[170],files[190],files[200],files[210],files[220],files[230],files[240],files[250],files[260],files[270],files[275]]
fff

['./archive/0819_UkraineCombinedTweetsDeduped.csv.gzip',
 './archive/0829_UkraineCombinedTweetsDeduped.csv.gzip',
 './archive/0918_UkraineCombinedTweetsDeduped.csv.gzip',
 './archive/1028_UkraineCombinedTweetsDeduped.csv.gzip',
 './archive/1117_UkraineCombinedTweetsDeduped.csv.gzip',
 './archive/1127_UkraineCombinedTweetsDeduped.csv.gzip',
 './archive/UkraineWar\\UkraineWar\\0403_UkraineCombinedTweetsDeduped.csv.gzip',
 './archive/UkraineWar\\UkraineWar\\0413_UkraineCombinedTweetsDeduped.csv.gzip',
 './archive/UkraineWar\\UkraineWar\\0423_UkraineCombinedTweetsDeduped.csv.gzip',
 './archive/UkraineWar\\UkraineWar\\0515_UkraineCombinedTweetsDeduped.csv.gzip',
 './archive/UkraineWar\\UkraineWar\\0604_UkraineCombinedTweetsDeduped.csv.gzip',
 './archive/UkraineWar\\UkraineWar\\0626_UkraineCombinedTweetsDeduped.csv.gzip',
 './archive/UkraineWar\\UkraineWar\\0706_UkraineCombinedTweetsDeduped.csv.gzip',
 './archive/UkraineWar\\UkraineWar\\0716_UkraineCombinedTweetsDeduped.csv.gzip',
 './archiv

In [66]:
data = []
for f in fff:
    df = pd.read_csv(f, compression='gzip', index_col=0,encoding='utf-8', quoting=csv.QUOTE_ALL)
    df = df[(df['language']=='en')&(~df['location'].isin(['Ukraine','Kyiv, Ukraine','Україна','Kyiv','Украина']))]\
    [['location','text','extractedts','hashtags']]
    data.append(df)
    print(f)

len(data)

./archive/0819_UkraineCombinedTweetsDeduped.csv.gzip
./archive/0829_UkraineCombinedTweetsDeduped.csv.gzip
./archive/0918_UkraineCombinedTweetsDeduped.csv.gzip
./archive/1028_UkraineCombinedTweetsDeduped.csv.gzip
./archive/1117_UkraineCombinedTweetsDeduped.csv.gzip
./archive/1127_UkraineCombinedTweetsDeduped.csv.gzip
./archive/UkraineWar\UkraineWar\0403_UkraineCombinedTweetsDeduped.csv.gzip
./archive/UkraineWar\UkraineWar\0413_UkraineCombinedTweetsDeduped.csv.gzip
./archive/UkraineWar\UkraineWar\0423_UkraineCombinedTweetsDeduped.csv.gzip
./archive/UkraineWar\UkraineWar\0515_UkraineCombinedTweetsDeduped.csv.gzip
./archive/UkraineWar\UkraineWar\0604_UkraineCombinedTweetsDeduped.csv.gzip
./archive/UkraineWar\UkraineWar\0626_UkraineCombinedTweetsDeduped.csv.gzip
./archive/UkraineWar\UkraineWar\0706_UkraineCombinedTweetsDeduped.csv.gzip
./archive/UkraineWar\UkraineWar\0716_UkraineCombinedTweetsDeduped.csv.gzip
./archive/UkraineWar\UkraineWar\0726_UkraineCombinedTweetsDeduped.csv.gzip
./archi

21

In [67]:
df = pd.concat(data, axis=0) 
df.shape

(2967708, 4)

In [68]:
df[:5]

,location,text,extractedts,hashtags
0,Afrika Borwa,Dear vaccine advocate\n\nDo take the COVID19 m...,2022-08-19 08:07:26.836769,"[{'text': 'Pfizer', 'indices': [189, 196]}, {'..."
5,NaN,👇 Good news you may have missed: First @WFP sh...,2022-08-19 05:12:06.096707,"[{'text': 'humanitarian', 'indices': [66, 79]}..."
6,NYC,Opinion: The Ukraine war is also being fought ...,2022-08-19 08:02:53.510769,"[{'text': 'Ukraine', 'indices': [68, 76]}, {'t..."
8,NaN,"Ethereum's spot setup looks grim, but derivati...",2022-08-19 05:12:06.018650,"[{'text': 'cryptocurrency', 'indices': [79, 94..."
10,United States,🛢 Today's Crude Oil Prices:\r📈 WTI: $90.63 (+0...,2022-08-19 08:04:59.090331,"[{'text': 'putin', 'indices': [93, 99]}, {'tex..."


In [48]:
# tag = df[:10].to_dict('records')
# for t in tag:
#     print(t['text'])
#     print('_____________________________')
#     # for t2 in t['hashtags']:
#     #     print(t2)

In [70]:
# df['location'].value_counts()[:30]

In [71]:
# print(df.shape)
# df = df[df['location']=='Germany']
# print(df.shape)


In [72]:
# def clean_text(text): 
#     stopwords = ["a", "about", "above", "after", "again", "against", "all", "am", "an", \
#                  "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", \
#                  "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", \
#                  "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", \
#                  "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", \
#                  "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", \
#                  "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", \
#                  "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", \
#                  "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", \
#                  "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", \
#                  "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", \
#                  "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", \
#                  "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", \
#                  "your", "yours", "yourself", "yourselves" ]
#     text = text.lower()
#     words = text.split()
#     no_words = [w for w in words if w not in stopwords]
#     text = " ".join(no_words)
    
#     text = re.sub(r'RT @\w+: ','', text)
# #     text = re.sub(r'-.+','', text)
# #     text = re.sub('<[^>]*>', '', text)
#     text = re.sub(r'(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)','', text)
# #     text = re.sub(r'(?<!\S)\d+(?!\S)',' ', text)
# #     text = re.sub(r'\s\\s\\s','\s', text)
# #     text = re.sub(r'/\s/\s','\s', text)
# #     text = ' '.join(list(filter(lambda t: len(t)>1, text.split()) ))
# #     text = re.sub(r'\sлот\s|аналог|\sабо\s|\sдк\s|\sдля\s|\sта\s|\sпо\s|\sна\s|\sдо\s|\sза\s|\sкод|\sдсту|еквівалент|ін\s|набір|нк\s|cpv',' ', text).replace('  ',' ')
#     return text

# # df['title'] = df['title'].astype(str)
# df['text2'] = df['text'].apply(clean_text)
# # print(df['title'][2])


In [73]:
# for i,t in df[:30].iterrows():
#     print(t['text'])
#     print('---------------------')
    

In [121]:
df.shape

(2967708, 4)

# #IStandWithPutin or #IStandWithRussia


In [166]:
bad = df[df['hashtags'].str.contains('IStandWithPutin|NaziUkraine|ZelenskyWarCriminal|Istandwithrussia|\
ZelenskyWarCriminal|UkraineTerrorState|StopNaziUkraine|StopNATO', regex=True)].reset_index()
bad.shape

(9299, 5)

In [167]:
bad['label'] = 1
bad

,index,location,text,extractedts,hashtags,label
0,432,India,@WIONews #Ukrainian Arms exported to #Pakistan...,2022-08-19 07:51:46.650272,"[{'text': 'Ukrainian', 'indices': [9, 19]}, {'...",1
1,459,India,@Anand11112293 @WIONews #Ukrainian Arms export...,2022-08-19 07:51:46.137381,"[{'text': 'Ukrainian', 'indices': [24, 34]}, {...",1
2,1295,NaN,"Things That Make You Go, ""Hmmmmm?"" - #UkraineR...",2022-08-19 11:22:22.145495,"[{'text': 'UkraineRussiaWar', 'indices': [37, ...",1
3,1844,USA,@ABCWorldNews @BrittClennett Pump Up The #FEAR...,2022-08-19 09:37:16.577863,"[{'text': 'FEAR', 'indices': [41, 46]}, {'text...",1
4,2367,NaN,"@MarkLmao80 @geraldcelente @ZelenskyyUa Bro, a...",2022-08-19 06:42:14.592350,"[{'text': 'Ukraine', 'indices': [70, 78]}, {'t...",1
...,...,...,...,...,...,...
9294,357282,NaN,@WorldAffairsPro @paulanewtonCNN @iclgi_networ...,2022-03-30 00:09:34.836217,"[{'text': 'Putin', 'indices': [607, 613]}, {'t...",1
9295,358736,NaN,#Ukraine: #Azov base near #Mariupol. #Nazi fla...,2022-03-30 00:09:31.600017,"[{'text': 'Ukraine', 'indices': [0, 8]}, {'tex...",1
9296,360627,"Great Amwell, East",In order to understand where Nazis in Ukraine ...,2022-03-29 23:54:38.951990,"[{'text': 'NaziUkraine', 'indices': [256, 268]...",1
9297,361073,NaN,#Biden is hinting at involving U.S. troops in ...,2022-03-30 00:00:55.511967,"[{'text': 'Biden', 'indices': [20, 26]}, {'tex...",1


In [85]:
# test.loc[43]['text']

In [168]:
bad

,index,location,text,extractedts,hashtags,label
0,432,India,@WIONews #Ukrainian Arms exported to #Pakistan...,2022-08-19 07:51:46.650272,"[{'text': 'Ukrainian', 'indices': [9, 19]}, {'...",1
1,459,India,@Anand11112293 @WIONews #Ukrainian Arms export...,2022-08-19 07:51:46.137381,"[{'text': 'Ukrainian', 'indices': [24, 34]}, {...",1
2,1295,NaN,"Things That Make You Go, ""Hmmmmm?"" - #UkraineR...",2022-08-19 11:22:22.145495,"[{'text': 'UkraineRussiaWar', 'indices': [37, ...",1
3,1844,USA,@ABCWorldNews @BrittClennett Pump Up The #FEAR...,2022-08-19 09:37:16.577863,"[{'text': 'FEAR', 'indices': [41, 46]}, {'text...",1
4,2367,NaN,"@MarkLmao80 @geraldcelente @ZelenskyyUa Bro, a...",2022-08-19 06:42:14.592350,"[{'text': 'Ukraine', 'indices': [70, 78]}, {'t...",1
...,...,...,...,...,...,...
9294,357282,NaN,@WorldAffairsPro @paulanewtonCNN @iclgi_networ...,2022-03-30 00:09:34.836217,"[{'text': 'Putin', 'indices': [607, 613]}, {'t...",1
9295,358736,NaN,#Ukraine: #Azov base near #Mariupol. #Nazi fla...,2022-03-30 00:09:31.600017,"[{'text': 'Ukraine', 'indices': [0, 8]}, {'tex...",1
9296,360627,"Great Amwell, East",In order to understand where Nazis in Ukraine ...,2022-03-29 23:54:38.951990,"[{'text': 'NaziUkraine', 'indices': [256, 268]...",1
9297,361073,NaN,#Biden is hinting at involving U.S. troops in ...,2022-03-30 00:00:55.511967,"[{'text': 'Biden', 'indices': [20, 26]}, {'tex...",1


In [172]:
good = df[df['hashtags'].str.contains('RussiaIsATerroristState|StandWithUkraine|PutinWarCriminal|\
StopRussiaNOW|StopRussia|StopPutin|CrimeaIsUkraine|FreeUkraine|UkraineUnderAttaсk', regex=True)].reset_index()
good.shape

(197094, 5)

In [153]:
# df.shape

In [173]:
good = good.sample(frac=0.30)
print(good.shape)


(59128, 5)


In [174]:
good['label'] = 0
good = good.reset_index()
good

,level_0,index,location,text,extractedts,hashtags,label
0,168299,6243779,NaN,important to be human 😉 ❤️\n#Army #Zelensky #U...,2022-03-13 22:17:51.478278,"[{'text': 'Army', 'indices': [45, 50]}, {'text...",0
1,113619,23753,NaN,Chicago. Meeting of wounded Ukrainian fighters...,2022-07-26 09:24:24.458596,"[{'text': 'StandWithUkraine', 'indices': [147,...",0
2,18139,55964,日本,🇺🇦Ukrainian army is trying to save the animals...,2022-04-03 04:22:00.717577,"[{'text': 'StandWithUkraine', 'indices': [63, ...",0
3,78307,85720,NaN,A woman sits and silently cries near the Kyiv ...,2022-06-26 11:57:51.671206,"[{'text': 'RussianUkrainianWar', 'indices': [1...",0
4,61141,14196,"Hastings, NE",Let's visualize our future 🇺🇦\n#StandWithUkrai...,2022-05-15 00:58:14.116459,"[{'text': 'StandWithUkraine', 'indices': [44, ...",0
...,...,...,...,...,...,...,...
59123,58445,246269,Georgia,"“As soon as we have more weapons, we will go a...",2022-04-23 18:45:07.824515,"[{'text': 'territories', 'indices': [68, 80]},...",0
59124,147627,5840375,"New York, NY",People from around the world keep singing Ukra...,2022-03-13 01:16:13.061487,"[{'text': 'StandWithUkraine', 'indices': [91, ...",0
59125,88237,9679,"North Carolina, USA",@Mcatnip @IamMultiversal @St2Naudrey @RzPtz @R...,2022-07-06 04:18:27.153846,"[{'text': 'StrongerTogether2022', 'indices': [...",0
59126,100741,41765,Europe,Kyrylo was 8. Russia’s missiles killed him in ...,2022-07-16 09:14:43.669043,"[{'text': 'StopRussia', 'indices': [109, 120]}...",0


In [159]:
# good['text'].to_list()

In [175]:
total = good.append(bad)
total = total[['label','text']]
total.shape

(68427, 2)

In [176]:
total.to_csv('total.csv', index=False) 


In [31]:
# bad.to_excel('bad.xlsx', index=False)

In [165]:
total

,label,text
0,0,Adam Kaufman: Don’t know about you but I’m wil...
1,0,@DefenceU Russians will say they have targeted...
2,0,Every day that passes #Russia wages an illegal...
3,0,In the Kharkiv metro animators calm down child...
4,0,@Sjsu111 please RT if you think #Putin is terr...
...,...,...
9294,1,@WorldAffairsPro @paulanewtonCNN @iclgi_networ...
9295,1,#Ukraine: #Azov base near #Mariupol. #Nazi fla...
9296,1,In order to understand where Nazis in Ukraine ...
9297,1,#Biden is hinting at involving U.S. troops in ...


In [180]:
bad['date'] = pd.to_datetime(bad['extractedts'])
bad.dtypes

index                   int64
location               object
text                   object
extractedts            object
hashtags               object
label                   int64
date           datetime64[ns]
dtype: object

In [184]:
bad.groupby(pd.Grouper(key='date',freq='M'))['text'].agg(['count'])

,count
date,
2022-03-31,7089
2022-04-30,484
2022-05-31,174
2022-06-30,176
2022-07-31,232
2022-08-31,590
2022-09-30,36
2022-10-31,43
2022-11-30,475


In [188]:
bad[bad['date']>'2022-11-01']['hashtags'].to_list()

["[{'text': 'Warzone2', 'indices': [134, 143]}, {'text': 'FTXScandal', 'indices': [144, 155]}, {'text': 'FTX', 'indices': [156, 160]}, {'text': 'FTXUkraine', 'indices': [161, 172]}, {'text': 'FTXScam', 'indices': [173, 181]}, {'text': 'FTXbiden', 'indices': [182, 191]}, {'text': 'Ukraine', 'indices': [192, 200]}, {'text': 'PolandMissileStrike', 'indices': [201, 221]}, {'text': 'Russia', 'indices': [222, 229]}, {'text': 'Putin', 'indices': [230, 236]}, {'text': 'Biden', 'indices': [237, 243]}, {'text': 'G20Summit2022', 'indices': [244, 258]}, {'text': 'ZelenskyWarCriminal', 'indices': [259, 279]}]",
 "[{'text': 'Zelinsky', 'indices': [123, 132]}, {'text': 'Ukraine', 'indices': [133, 141]}, {'text': 'ZelenskyWarCriminal', 'indices': [142, 162]}]",
 "[{'text': 'ukraine', 'indices': [47, 55]}, {'text': 'ZelenskyWarCriminal', 'indices': [56, 76]}, {'text': 'Poland', 'indices': [77, 84]}]",
 "[{'text': 'Ukraine', 'indices': [13, 21]}, {'text': 'ZelenskyWarCriminal', 'indices': [33, 53]}]",
 